In [ ]:
# | default_exp integraion_tests

In [ ]:
from airt.testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.


2023-01-20 10:13:28.388550: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[INFO] numexpr.utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO] numexpr.utils: NumExpr defaulting to 8 threads.
[WARNING] airt.testing.activate_by_import: Failed to set gpu memory limit for tf; This could happen because of no gpu availability


In [ ]:
# | export

import json
import os
import random
import string
import time
from datetime import datetime, timedelta
from typing import *

import httpx
import pandas as pd
import pyotp
from azure.identity import DefaultAzureCredential
from azure.mgmt.storage import StorageManagementClient
from fastcore.script import call_parse, Param
from sqlmodel import select

from airt_service.sanitizer import sanitized_print
from airt_service.aws.utils import upload_to_s3_with_retry
from airt.remote_path import RemotePath

In [ ]:
import contextlib
import pytest
import threading
import uvicorn
from pathlib import Path
import posix_ipc

from airt_service.helpers import set_env_variable_context
from airt_service.server import create_ws_server

[INFO] airt.executor.subcommand: Module loaded.


In [ ]:
# | export


def integration_scenario_docs(base_url: str = "http://127.0.0.1:6006"):
    """Test fastapi docs

    Args:
        base_url: Base url
    """
    sanitized_print("getting /docs")
    r = httpx.get(f"{base_url}/docs")
    assert not r.is_error, r  # nosec B101

    sanitized_print("getting /redocs")
    r = httpx.get(f"{base_url}/redoc")
    assert not r.is_error, r  # nosec B101

In [ ]:
# | export


def test_auth(base_url: str, username: str, password: str) -> str:
    """Get jwt token for given credentials

    Args:
        base_url: Base url
        username: Username
        password: Password
    Returns:
        The jwt token for the given username and password
    """
    # Authenticate
    sanitized_print("authenticating and getting token")
    r = httpx.post(
        f"{base_url}/token",
        data=dict(username=username, password=password),
    )
    assert not r.is_error, r.text  # nosec B101
    token = r.json()["access_token"]
    return token

In [ ]:
# | export


def test_create_user(base_url: str) -> Tuple[Dict[str, Any], str]:
    """Create a new user for testing

    Args:
        base_url: Base url
    Returns:
        The user dictionary and its password as a tuple
    """
    # Get token for super user
    token = os.environ["AIRT_SERVICE_TOKEN"]
    headers = {"Authorization": f"Bearer {token}"}

    sanitized_print("creating user")
    username = "".join(  # nosec
        random.choice(string.ascii_lowercase) for _ in range(10)
    )
    password = "".join(  # nosec
        random.choice(string.ascii_lowercase) for _ in range(10)
    )
    r = httpx.post(
        f"{base_url}/user/",
        json=dict(
            username=username,
            first_name="integration",
            last_name="user",
            email=f"{username}@email.com",
            subscription_type="small",
            super_user=False,
            password=password,
            otp=None,
        ),
        headers=headers,
        timeout=30,
    )
    assert not r.is_error, r.text  # nosec B101
    user = r.json()
    return user, password

In [ ]:
# | export


def test_apikey(
    base_url: str, headers: Dict[str, str], otp: Optional[str] = None
) -> str:
    """Create apikey for testing

    Args:
        base_url: Base url
        headers: Headers dict with authorization header
        otp: Dynamically generated six-digit verification code from the authenticator app
    Returns:
        The apikey jwt token
    """
    sanitized_print("creating apikey")
    r = httpx.post(
        f"{base_url}/apikey",
        json=dict(
            expiry=(datetime.utcnow() + timedelta(minutes=60)).isoformat(), otp=otp
        ),
        headers=headers,
    )
    assert not r.is_error, r.text  # nosec B101
    apikey = r.json()["access_token"]
    return apikey

In [ ]:
# | export


def check_steps_completed(url: str, headers: Dict[str, str]) -> Dict[str, Any]:
    """Check whether completed steps equals to total steps

    Args:
        url: Url to call
        headers: Headers dict with authorization header
    Returns:
        The dictionary returned by url
    """
    sanitized_print("start waiting for steps to complete")
    while True:
        r = httpx.get(url, headers=headers)
        assert not r.is_error, f"{r.text=} {r.status_code=}"  # nosec B101
        obj = r.json()
        if obj["completed_steps"] == obj["total_steps"]:
            break
        time.sleep(5)
    sanitized_print("stop waiting for steps to complete")
    return obj

In [ ]:
# | export


def test_csv_local_datablob_and_datasource(
    base_url: str, headers: Dict[str, str]
) -> Tuple[Dict[str, Any], Dict[str, Any]]:
    """Create datablob from local, upload csv files and create datasource from it

    Args:
        base_url: Base url
        headers: Headers dict with authorization header
    Returns:
        Datablob and datasource dictionaries as a tuple
    """
    # Create csv datablob
    sanitized_print("creating datablob")
    r = httpx.post(
        f"{base_url}/datablob/from_local/start",
        json=dict(path="tmp/test-folder/"),
        headers=headers,
    )
    assert not r.is_error, r.text  # nosec B101
    datablob_uuid = r.json()["uuid"]
    presigned = r.json()["presigned"]

    sanitized_print("downloading csv file")
    with RemotePath.from_url(
        remote_url=f"s3://test-airt-service/account_312571_events",
        pull_on_enter=True,
        push_on_exit=False,
        exist_ok=True,
        parents=False,
        access_key=os.environ["AWS_ACCESS_KEY_ID"],
        secret_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    ) as test_s3_path:
        df = pd.read_parquet(test_s3_path.as_path())
        df.to_csv(test_s3_path.as_path() / "file.csv", index=False)

        sanitized_print("uploading csv file using presigned url")
        upload_to_s3_with_retry(
            test_s3_path.as_path() / "file.csv", presigned["url"], presigned["fields"]
        )

    # Create datasource from csv datablob
    sanitized_print("creating datasource")
    r = httpx.post(
        f"{base_url}/datablob/{datablob_uuid}/to_datasource",
        json=dict(
            file_type="csv",
            deduplicate_data=True,
            index_column="PersonId",
            sort_by="OccurredTime",
            blocksize="256MB",
            kwargs_json=dict(
                usecols=[0, 1, 2, 3, 4],
                parse_dates=["OccurredTime"],
            ),
        ),
        headers=headers,
    )
    assert not r.is_error, r.text  # nosec B101
    datasource = r.json()

    # Wait for pull to complete
    datasource = check_steps_completed(
        url=f"{base_url}/datasource/{datasource['uuid']}", headers=headers
    )
    sanitized_print("pull completed for datasource")

    # Get datablob object to return
    datablob = check_steps_completed(
        url=f"{base_url}/datablob/{datablob_uuid}", headers=headers
    )

    # Display head and dtypes
    r = httpx.get(f"{base_url}/datasource/{datasource['uuid']}/head", headers=headers)
    sanitized_print("head of datasource")
    sanitized_print(r.json())
    r = httpx.get(f"{base_url}/datasource/{datasource['uuid']}/dtypes", headers=headers)
    sanitized_print("dtypes of datasource")
    sanitized_print(r.json())

    return datablob, datasource

In [ ]:
# | export


def test_azure_datablob(base_url: str, headers: Dict[str, str]) -> Dict[str, Any]:
    """Create datablob using from_azure route

    Args:
        base_url: Base url
        headers: Headers dict with authorization header
    Returns:
        A azure datablob
    """
    storage_client = StorageManagementClient(
        DefaultAzureCredential(), os.environ["AZURE_SUBSCRIPTION_ID"]
    )
    keys = storage_client.storage_accounts.list_keys(
        "test-airt-service", "testairtservice"
    )
    credential = keys.keys[0].value

    # Create azure datablob
    sanitized_print("creating azure datablob")
    r = httpx.post(
        f"{base_url}/datablob/from_azure_blob_storage",
        json=dict(
            uri="https://testairtservice.blob.core.windows.net/test-container/account_312571_events",
            credential=credential,
            cloud_provider="azure",
            region="westeurope",
        ),
        headers=headers,
    )
    assert not r.is_error, r.text  # nosec B101
    datablob = r.json()

    # Wait for pull to complete
    datablob = check_steps_completed(
        url=f"{base_url}/datablob/{datablob['uuid']}", headers=headers
    )
    sanitized_print("pull completed for azure datablob")

    return datablob

In [ ]:
# | export


def test_model(
    base_url: str, headers: Dict[str, str], datasource: Dict[str, Any]
) -> Dict[str, Any]:
    """Train model and evaluate it for testing

    Args:
        base_url: Base url
        headers: Headers dict with authorization header
        datasource: Datasource dictionary
    Returns:
        The model dictionary
    """
    # Train model
    sanitized_print("training model")
    r = httpx.post(
        f"{base_url}/model/train",
        json=dict(
            data_uuid=datasource["uuid"],
            client_column="AccountId",
            target_column="DefinitionId",
            target="load*",
            predict_after=20 * 24 * 60 * 60,
        ),
        headers=headers,
    )
    assert not r.is_error, r.text  # nosec B101
    model = r.json()

    # Wait for model training to complete
    model = check_steps_completed(
        url=f"{base_url}/model/{model['uuid']}", headers=headers
    )
    sanitized_print("model training completed")

    # Evaluate model
    r = httpx.get(f"{base_url}/model/{model['uuid']}/evaluate", headers=headers)
    assert not r.is_error  # nosec B101
    sanitized_print("model evaluation")
    sanitized_print(r.json())

    return model

In [ ]:
# | export


def test_prediction(
    base_url: str, headers: Dict[str, str], model: Dict[str, Any]
) -> Dict[str, Any]:
    """Run prediction and evaluate prediction for testing

    Args:
        base_url: Base url
        headers: Headers dict with authorization header
        model: Model dictionary
    Returns:
        The prediction dictionary
    """
    # Run prediction for the model
    sanitized_print("running prediction")
    r = httpx.post(
        f"{base_url}/model/{model['uuid']}/predict",
        headers=headers,
    )
    assert not r.is_error, r.text  # nosec B101
    prediction = r.json()

    # Wait for prediction to complete
    prediction = check_steps_completed(
        url=f"{base_url}/prediction/{prediction['uuid']}", headers=headers
    )
    sanitized_print("prediction completed")

    # Get prediction as pandas
    r = httpx.get(f"{base_url}/prediction/{prediction['uuid']}/pandas", headers=headers)
    assert not r.is_error  # nosec B101
    sanitized_print("prediction as pandas")
    sanitized_print(r.json())

    return prediction

In [ ]:
# | export


def test_generate_mfa_url(base_url: str, headers: Dict[str, str]) -> Dict[str, Any]:
    """Generate mfa provisioning uri

    Args:
        base_url: Base url
        headers: Headers dict with authorization header

    Returns:
        The provisioning uri generated from the secret
    """

    r = httpx.get(f"{base_url}/user/mfa/generate", headers=headers)
    assert not r.is_error, f"{r.text=} {r.status_code=}"  # nosec B101
    sanitized_print("Generating mfa url")
    return r.json()

In [ ]:
# | export


def get_valid_otp(mfa_url: str) -> str:
    """Get valid otp for the mfa_url

    Args:
        mfa_url: mfa provisioning url

    Returns:
        The valid otp for the url
    """
    return pyotp.TOTP(pyotp.parse_uri(mfa_url).secret).now()

In [ ]:
# | export


def test_activate_mfa(
    base_url: str, mfa_url: str, headers: Dict[str, str]
) -> Dict[str, Any]:
    """Activate mfa

    Args:
        base_url: Base url
        mfa_url: mfa provisioning url
        headers: Headers dict with authorization header

    Returns:
        The provisioning uri generated from the secret
    """
    r = httpx.post(
        f"{base_url}/user/mfa/activate",
        json=dict(user_otp=get_valid_otp(mfa_url)),
        headers=headers,
    )
    assert not r.is_error, f"{r.text=} {r.status_code=}"  # nosec B101
    sanitized_print("Activate mfa")
    return r.json()

In [ ]:
# | export


def reset_test_user_password(
    base_url: str, headers: Dict[str, str], username: str, password: str, otp: str
):
    """Reset the test user password"""
    sanitized_print(f"Resetting password for: {username}")
    r = httpx.post(
        f"{base_url}/user/reset_password",
        json=dict(username=username, new_password=password, otp=otp),
    )
    assert not r.is_error, r.text  # nosec B101
    sanitized_print(r.text)

In [ ]:
# | export


def test_disable_mfa(
    base_url: str, headers: Dict[str, str], username: str, otp: Optional[str] = None
):
    """Disable MFA for the user"""
    current_active_user = httpx.get(
        f"{base_url}/user/details?user_id_or_name=None", headers=headers
    )
    current_active_user_uuid = current_active_user.json()["uuid"]
    r = httpx.delete(
        f"{base_url}/user/mfa/{current_active_user_uuid}/disable?otp={otp}",
        headers=headers,
    )
    assert not r.is_error, r.text  # nosec B101
    sanitized_print(r.text)
    assert username in r.text  # nosec B101
    sanitized_print("Deactivate mfa")

In [ ]:
# | export


def test_auth_with_otp(
    base_url: str, username: str, password: str, mfa_url: str, retry_limit: int = 3
) -> str:
    """Get jwt token for given credentials and otp

    Args:
        base_url: Base url
        username: Username
        password: Password
        mfa_url: MFA URL
        retry_limit: Retry limit if there is an error with otp auth
    Returns:
        The jwt token for the given username and password
    """
    # Authenticate
    sanitized_print("authenticating with otp and getting token")
    for i in range(retry_limit):
        otp = get_valid_otp(mfa_url)
        r = httpx.post(
            f"{base_url}/token",
            data=dict(
                username=username,
                password=json.dumps(
                    {
                        "password": password,
                        "user_otp": otp,
                    }
                ),
            ),
        )

        if not r.is_error:
            break

    assert not r.is_error, r.text  # nosec B101
    token = r.json()["access_token"]
    return token

In [ ]:
# | export


def delete_test_user(base_url: str, test_username: str):
    """Delete the test user created for testing

    Args:
        base_url: Base url
        test_username: Username to delete
    """
    # Get token for super user
    token = os.environ["AIRT_SERVICE_TOKEN"]
    headers = {"Authorization": f"Bearer {token}"}

    sanitized_print("deleting test user")
    r = httpx.post(
        f"{base_url}/user/cleanup",
        json=dict(
            username=test_username,
        ),
        headers=headers,
        timeout=None,
    )
    assert not r.is_error, r.text  # nosec B101

In [ ]:
# | export


def integration_tests(base_url: str = "http://127.0.0.1:6006"):
    """Integration tests

    Args:
        base_url: Base url
    """
    sanitized_print("starting integration tests")
    integration_scenario_docs(base_url)

    user, password = test_create_user(base_url)

    token = test_auth(
        base_url,
        username=user["username"],
        password=password,
    )
    headers = {"Authorization": f"Bearer {token}"}

    # enable mfa for the user
    mfa_url = test_generate_mfa_url(base_url, headers)
    # activate mfa
    test_activate_mfa(base_url, mfa_url["mfa_url"], headers)

    # Get token by passing password and otp as json encoded dict
    token = test_auth_with_otp(
        base_url,
        username=user["username"],
        password=password,
        mfa_url=mfa_url["mfa_url"],
        retry_limit=3,
    )

    headers = {"Authorization": f"Bearer {token}"}

    apikey = test_apikey(base_url, headers, otp=get_valid_otp(mfa_url["mfa_url"]))
    headers = {"Authorization": f"Bearer {apikey}"}

    datablob, datasource = test_csv_local_datablob_and_datasource(base_url, headers)

    azure_datablob = test_azure_datablob(base_url, headers)

    model = test_model(base_url, headers, datasource)

    prediction = test_prediction(base_url, headers, model)

    new_password = "new_password"  # nosec B105
    reset_test_user_password(
        base_url=base_url,
        headers=headers,
        username=user["username"],
        password=new_password,
        otp=get_valid_otp(mfa_url["mfa_url"]),
    )

    # Get token by using the new password
    token = test_auth_with_otp(
        base_url,
        username=user["username"],
        password=new_password,
        mfa_url=mfa_url["mfa_url"],
        retry_limit=3,
    )

    headers = {"Authorization": f"Bearer {token}"}

    test_disable_mfa(
        base_url, headers, user["username"], otp=get_valid_otp(mfa_url["mfa_url"])
    )

    delete_test_user(base_url, test_username=user["username"])

    sanitized_print("ok")

In [ ]:
# from https://github.com/encode/uvicorn/issues/742


def test_integration_tests():
    # Start integration tests
    token = test_auth(
        "http://127.0.0.1:6006",
        username="kumaran",
        password=os.environ["AIRT_SERVICE_SUPER_USER_PASSWORD"],
    )
    with set_env_variable_context(variable="AIRT_SERVICE_TOKEN", value=token):
        integration_tests()


class Server(uvicorn.Server):
    def install_signal_handlers(self):
        pass

    @contextlib.contextmanager
    def run_in_thread(self):
        thread = threading.Thread(target=self.run)
        thread.start()
        try:
            while not self.started:
                time.sleep(1e-3)
            yield
        finally:
            self.should_exit = True
            thread.join()


with posix_ipc.Semaphore(
    "/infobip_kafka_topics_semaphore", flags=posix_ipc.O_CREAT, initial_value=1
) as sem:
    with set_env_variable_context(variable="JOB_EXECUTOR", value="fastapi"):
        app, fast_kafka_api_app = create_ws_server(
            assets_path=Path("../assets"), start_process_for_username=None
        )
        config = uvicorn.Config(app, host="127.0.0.1", port=6006, log_level="info")
        server = Server(config=config)

        with server.run_in_thread():
            # Server started.
            sanitized_print("server started")

            test_integration_tests()

        sanitized_print("server stopped")
        # Server stopped.

[INFO] airt_service.server: kafka_config={'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092', 'group_id': 'kumaran-airt-service-kafka-1:9092_group', 'auto_offset_reset': 'earliest'}


INFO:     Started server process [71801]
INFO:     Waiting for application startup.


[INFO] fast_kafka_api._components.asyncapi: Keeping the old async specifications at: 'asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api._components.asyncapi: Skipping generating async documentation in '/work/airt-service/notebooks/asyncapi/docs'
[INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
[INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
[INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
[INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fast_kafka_api._components.aiokafka_consumer

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:6006 (Press CTRL+C to quit)


server started
authenticating and getting token
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'infobip_training_data'})
[INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'infobip_training_data'}
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'infobip_realtime_data'})
[INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'infobip_realtime_data'}
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_st

...

[INFO] airt_service.background_task: Background task stderr for command: 'azure_blob_storage_pull 197':

[INFO] airt_service.background_task: Background task finished for command: 'azure_blob_storage_pull 197' with return code 0
INFO:     127.0.0.1:59736 - "GET /datablob/f3e83987-74ac-4c6a-8751-20ca01fcdd65 HTTP/1.1" 200 OK
stop waiting for steps to complete
pull completed for azure datablob
training model
INFO:     127.0.0.1:59750 - "POST /model/train HTTP/1.1" 200 OK
start waiting for steps to complete
INFO:     127.0.0.1:59762 - "GET /model/2e98d357-253a-4450-a34f-24d0c41bf624 HTTP/1.1" 200 OK
stop waiting for steps to complete
model training completed
INFO:     127.0.0.1:59776 - "GET /model/2e98d357-253a-4450-a34f-24d0c41bf624/evaluate HTTP/1.1" 200 OK
model evaluation
{'accuracy': 0.985, 'recall': 0.962, 'precision': 0.934}
running prediction
[INFO] airt_service.batch_job: create_batch_job(): command='predict 55', task='csv_processing'
[INFO] airt_service.batch_job_components.base

%4|1674209771.801|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1674209771.801|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
INFO:     Shutting down
INFO:     Waiting for application shutdown.


ok
[INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.


INFO:     Application shutdown complete.
INFO:     Finished server process [71801]


server stopped


In [ ]:
# | export


@call_parse
def run_integration_tests(
    host: Param("hostname", str),  # type: ignore
    port: Param("port", int),  # type: ignore
    protocol: Param("http or https", str) = "https",  # type: ignore
):
    """Run integration tests against given host and port

    Args:
        host: Hostname of the webserver to run tests against
        port: Port of the webserver
        protocol: Protocol to use for testing
    """
    base_url = f"{protocol}://{host}:{port}"
    integration_tests(base_url=base_url)

In [ ]:
def test_run_integration_tests(host, port, protocol):
    # Start integration tests
    token = test_auth(
        f"{protocol}://{host}:{port}",
        username="kumaran",
        password=os.environ["AIRT_SERVICE_SUPER_USER_PASSWORD"],
    )
    with set_env_variable_context(variable="AIRT_SERVICE_TOKEN", value=token):
        run_integration_tests(host, port, protocol)


with posix_ipc.Semaphore(
    "/infobip_kafka_topics_semaphore", flags=posix_ipc.O_CREAT, initial_value=1
) as sem:
    with set_env_variable_context(variable="JOB_EXECUTOR", value="fastapi"):
        app, fast_kafka_api_app = create_ws_server(
            assets_path=Path("../assets"), start_process_for_username=None
        )
        config = uvicorn.Config(app, host="127.0.0.1", port=6006, log_level="info")
        server = Server(config=config)

        with server.run_in_thread():
            test_run_integration_tests("127.0.0.1", port=6006, protocol="http")

        sanitized_print("server stopped")

[INFO] airt_service.server: kafka_config={'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092', 'group_id': 'kumaran-airt-service-kafka-1:9092_group', 'auto_offset_reset': 'earliest'}


INFO:     Started server process [71801]
INFO:     Waiting for application startup.


[INFO] fast_kafka_api._components.asyncapi: Keeping the old async specifications at: 'asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api._components.asyncapi: Skipping generating async documentation in '/work/airt-service/notebooks/asyncapi/docs'
[INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
[INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
[INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
[INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fast_kafka_api._components.aiokafka_consumer

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:6006 (Press CTRL+C to quit)


authenticating and getting token
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'infobip_start_training_data'})
[INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'infobip_start_training_data'}
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'infobip_training_data'})
[INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'infobip_training_data'}
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_state

...

ok


INFO:     Waiting for application shutdown.


[INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.


INFO:     Application shutdown complete.
INFO:     Finished server process [71801]


server stopped
